# Live Streaming

In [1]:
!pip install gradio -q
!pip install ultralytics -q
!pip install supervision -q

In [2]:
from PIL import Image
from tqdm.notebook import tqdm
import gradio as gr
import supervision as sv
from ultralytics import YOLO
import numpy as np
import cv2  # for video to image conversion


In [3]:
import os

In [ ]:
def text_to_srt()-> bool:
  """
  Convert Text File into .srt file for caption

  Parameters
  ----------
  None

  Return
  ------
  bool
  """
  flag = os.system("python3 text_to_subtitles.py")
  if flag == 0:
    return True
  return False

In [ ]:
def merget_srt_to_video() -> bool:
    """
    Merge .srt file with video

    Parameters
    ----------
    None

    Return
    ------
    bool
    """
    query = "ffmpeg -i result.mp4 -vf subtitles=subtitles.srt output.mp4 -y"
    print(query)
    var = os.system(query)
    print(var)
    if var==0:
      return True
    return False


In [ ]:
from google.colab import files

In [ ]:
def export_video() -> None:
  """
  Download resultant video

  Parameters
  ----------
  None

  Return
  ------
  None
  """
  if text_to_srt():
    if merget_srt_to_video():
      files.download("output.mp4")


In [ ]:


model = YOLO('yolov8s.pt')
byte_tracker = sv.ByteTrack()
annotator = sv.BoxAnnotator()


In [ ]:
def process_live(frame):
    results = model(frame)[0]
    detections = sv.Detections.from_ultralytics(results)
    annotator = sv.BoxAnnotator()
    detections = byte_tracker.update_with_detections(detections)
    labels = [
         f"#{tracker_id} {model.model.names[class_id]} {confidence:0.2f}"
         for _, _, confidence, class_id, tracker_id
         in detections
    ]
    yield  annotator.annotate(scene=frame.copy(),
                               detections=detections, labels=labels)


In [ ]:
def process_video(SOURCE_VIDEO_PATH):
      """
      In this method video processing will be done.

      Parameters
      ----------
      SOURCE_VIDEO_PATH
        Video to detect track and count object

      Return
      ------
      Image
        Return the process frame .
      """
      # create BYTETracker instance
      byte_tracker = sv.ByteTrack()

      video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
      # create frame generator
      generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)
      # create LineCounter instance
      # create instance of BoxAnnotator and LineCounterAnnotator
      box_annotator = sv.BoxAnnotator(thickness=4, text_thickness=4, text_scale=2)
      print("Video Info :", video_info)
      # open target video file
          # loop over video frames
      for frame in tqdm(generator, total=video_info.total_frames):
          # model prediction on single frame and conversion to supervision Detections
        results = model(frame)
        # detections = sv.Detections.from_ultralytics(results)
        detections = sv.Detections(
          xyxy=results[0].boxes.xyxy.cpu().numpy(),
          confidence=results[0].boxes.conf.cpu().numpy(),
          class_id=results[0].boxes.cls.cpu().numpy().astype(int)
      )
        detections = byte_tracker.update_with_detections(detections)
        labels = [
            f"#{tracker_id} {model.model.names[class_id]} {confidence:0.2f}"
            for _, _, confidence, class_id, tracker_id
            in detections
        ]
      with open("subtitles.txt","a") as file:
        file.writelines(str(labels) + '\n' + '\n')
      ids = detections.class_id
      for id in ids:
        if object_count.count(id) == 0 :
          object_count.append(id)
      yield (box_annotator.annotate(frame=frame, detections=detections, labels=labels), len(object_count))





In [ ]:
title = "Object Detect Tracking and Counting"
with gr.Blocks(theme= gr.themes.Soft()) as io:
    with gr.Tab("Video Tracking") as record:
        gr.Markdown(f"<center><h1>{title}</h1></center>")
        with gr.Row():
            with gr.Column():
                input_image = gr.Video()

            with gr.Column():
                output_image = gr.Image()

        with gr.Row():
              total_count = gr.Textbox(label = "Number of Object")

        with gr.Row():
                input_button = gr.Button("Start Tracking")
                input_button.click(process_video, inputs=[input_image], outputs=[output_image, total_count])
        with gr.Row():
                input_button = gr.Button("Export Result")
                input_button.click(export_video, inputs= None, outputs = None)

    with gr.Tab("Live Tracking") as live:
        gr.Markdown(f"<center><h1>{title}</h1></center>")
        with gr.Row():
          with gr.Column():
             input_image = gr.Image(source='webcam', streaming=True)

          with gr.Column():
             output_image = gr.Image()
        with gr.Row():
              total_count = gr.Textbox(label = "Number of Object")
        with gr.Row():
             button  = gr.Button("Start Tracking")
             button.click(process_live, inputs=[input_image], outputs=[output_image])
        with gr.Row():
                input_button = gr.Button("Export Result")
                input_button.click(export_video, inputs= None, outputs = None)
io.queue()
io.launch(debug = True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://89a2d90d5a2b171aed.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Video Info : VideoInfo(width=640, height=360, fps=30, total_frames=318)


  0%|          | 0/318 [00:00<?, ?it/s]


0: 384x640 9 cars, 1 umbrella, 16.2ms
Speed: 3.9ms preprocess, 16.2ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 1 umbrella, 11.2ms
Speed: 2.2ms preprocess, 11.2ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 1 umbrella, 11.8ms
Speed: 1.8ms preprocess, 11.8ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 1 umbrella, 11.2ms
Speed: 1.9ms preprocess, 11.2ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 1 umbrella, 11.2ms
Speed: 1.9ms preprocess, 11.2ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 1 umbrella, 11.2ms
Speed: 1.9ms preprocess, 11.2ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 1 umbrella, 11.2ms
Speed: 1.2ms preprocess, 11.2ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 1 umbrella, 11.6ms
Speed:

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://0845dcc7eed69bcc77.gradio.live
Killing tunnel 127.0.0.1:7861 <> https://89a2d90d5a2b171aed.gradio.live


In [ ]:
!pip install torch==2.0.1

In [ ]:
!git clone https://github.com/sankalpvarshney/Track-And-Count-Object-using-YOLO.git
%cd Track-And-Count-Object-using-YOLO
#!conda create --prefix ./env python=3.8 -y
#conda activate ./env
!pip install ultralytics
!git clone https://github.com/ifzhang/ByteTrack.git
!cd ByteTrack
!sed -i 's/onnx==1.8.1/onnx==1.9.0/g' requirements.txt
!pip install -q -r requirements.txt
!python setup.py -q develop
!pip install -q cython-bbox
!pip install -q onemetric
!pip install -q loguru lap
!pip install numpy==1.22.4
!pip install supervision==0.1.0

Cloning into 'Track-And-Count-Object-using-YOLO'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 16 (delta 6), reused 10 (delta 3), pack-reused 0
Receiving objects: 100% (16/16), 4.48 KiB | 4.48 MiB/s, done.
Resolving deltas: 100% (6/6), done.
/content/Track-And-Count-Object-using-YOLO
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.4/617.4 kB 10.0 MB/s eta 0:00:00
Cloning into 'ByteTrack'...
remote: Enumerating objects: 2007, done.
remote: Total 2007 (delta 0), reused 0 (delta 0), pack-reused 2007
Receiving objects: 100% (2007/2007), 79.60 MiB | 29.28 MiB/s, done.
Resolving deltas: 100% (1141/1141), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 2.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour

In [ ]:
pip install --upgrade setuptools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.9/807.9 kB 6.4 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


In [ ]:
!pip install wheel setuptools pip --upgrade


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [ ]:
!pip3 install yolox --no-cache-dir

In [ ]:
%pwd

'/content'

In [ ]:
%cd /content/Track-And-Count-Object-using-YOLO

/content/Track-And-Count-Object-using-YOLO


In [ ]:
!python3 yolov8tracker.py -i "/content/out.avi" -o "/content/out1.avi"

Traceback (most recent call last):
  File "/content/Track-And-Count-Object-using-YOLO/yolov8tracker.py", line 1, in <module>
    from yolox.tracker.byte_tracker import BYTETracker, STrack
ModuleNotFoundError: No module named 'yolox'


In [ ]:
from yolov8tracker import TrackObject
obj = TrackObject("/content/out.avi","/content/out1.avi")
obj.process_video()

ImportError: ignored

In [ ]:
%cd ..

/content


In [ ]:
!git clone https://github.com/mohamedamine99/Object-tracking-and-counting-using-YOLOV8.git

Cloning into 'Object-tracking-and-counting-using-YOLOV8'...
remote: Enumerating objects: 249, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 249 (delta 27), reused 15 (delta 4), pack-reused 198
Receiving objects: 100% (249/249), 489.13 MiB | 31.55 MiB/s, done.
Resolving deltas: 100% (77/77), done.
Updating files: 100% (83/83), done.


In [ ]:
%cd /content/Object-tracking-and-counting-using-YOLOV8

/content/Object-tracking-and-counting-using-YOLOV8


In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110459 sha256=1e9753d3739658ce4f27eddfc33f2935dd611c2f1bd35f477653d682f2d0d779
  Stored in directory: /root/.cache/pip/wheels/0f/0c/ea/218f266af4ad626897562199fbbcba521b8497303200186102
Successfully built filterpy


In [ ]:
!git clone https://github.com/SkalskiP/yolov8-live.git

Cloning into 'yolov8-live'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 29 (delta 13), reused 14 (delta 4), pack-reused 0
Receiving objects: 100% (29/29), 6.18 KiB | 6.18 MiB/s, done.
Resolving deltas: 100% (13/13), done.


In [ ]:
%cd /content/yolov8-live

/content/yolov8-live


In [ ]:
!python3 -m venv venv


The virtual environment was not created successfully because ensurepip is not
available.  On Debian/Ubuntu systems, you need to install the python3-venv
package using the following command.

    apt install python3.10-venv

You may need to use sudo with that command.  After installing the python3-venv
package, recreate your virtual environment.

Failing command: /content/yolov8-live/venv/bin/python3



In [ ]:
!pip install -r requirements.txt -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.4/275.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 76.2 MB/s eta 0:00:00


In [ ]:
!python3 main.py

[ WARN:0@6.324] global cap_v4l.cpp:982 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@6.325] global obsensor_uvc_stream_channel.cpp:156 getStreamChannelGroup Camera index out of range
100% 83.7M/83.7M [00:05<00:00, 16.4MB/s]
2023-10-07 10:50:36.921264: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-07 10:50:37.900021: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Ultralytics YOLOv8.0.32 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8l summary (fused): 268 layers, 43668288 parameters, 0 gradients, 165.2 GFLOPs

Traceback (most recent call last):
  File "/content/yolov8-live/main.py", line 81, in <module>
    main()
  File "/content/yolov8-live/main.py", l

In [ ]:
!git lfs install
!git clone https://huggingface.co/spaces/ayoubkirouane/YoLo-Nas_Object-Detection_Tracking

Updated git hooks.
Git LFS initialized.
Cloning into 'YoLo-Nas_Object-Detection_Tracking'...
remote: Enumerating objects: 125, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (119/119), done.
remote: Total 125 (delta 16), reused 0 (delta 0), pack-reused 4
Receiving objects: 100% (125/125), 174.31 KiB | 901.00 KiB/s, done.
Resolving deltas: 100% (16/16), done.


In [ ]:
%cd /content/yolov8-live/YoLo-Nas_Object-Detection_Tracking

/content/yolov8-live/YoLo-Nas_Object-Detection_Tracking


In [ ]:
pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 684.5/684.5 kB 43.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 75.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.6/408.6 kB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 96.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 83.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 77.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.7 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/mohamedamine99/Object-tracking-and-counting-using-YOLOV8.git

Cloning into 'Object-tracking-and-counting-using-YOLOV8'...
remote: Enumerating objects: 249, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 249 (delta 27), reused 15 (delta 4), pack-reused 198
Receiving objects: 100% (249/249), 489.13 MiB | 32.54 MiB/s, done.
Resolving deltas: 100% (77/77), done.
Updating files: 100% (83/83), done.


In [ ]:
%cd /content/Object-tracking-and-counting-using-YOLOV8

/content/Object-tracking-and-counting-using-YOLOV8


In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 38.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110459 sha256=d7ac50e1f209659ec092a83bbf319414cabce11ce79f6a3a79bfa5f586321a00
  Stored in directory: /root/.cache/pip/wheels/0f/0c/ea/218f266af4ad626897562199fbbcba521b8497303200186102
  Created wheel for lap: filename=lap-0.4.0-cp310-cp310-linux_x86_64.whl size=1628963 sha256=f3e46fba672d6317f08b7437feaa3ed73ba2e7be673d4578761b9f37da7de1ca
  Stored in directory: /root/.cache/pip/wheels/00/42/2e/9dfe19270eea279d79e84767ff0d7b8082c3bf776cad00e83d
Successfully built filterpy lap


In [ ]:
!pip install -q ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 618.9/618.9 kB 9.6 MB/s eta 0:00:00


In [ ]:
!python3 yolo_detect_and_count.py

Ultralytics YOLOv8.0.195 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 27.4/78.2 GB disk)


In [ ]:


import numpy as np
import random
import os
import cv2
import sort

import time

# Commented out IPython magic to ensure Python compatibility.
# %pip install ultralytics
import ultralytics
ultralytics.checks()
from ultralytics import YOLO

class YOLOv8_ObjectDetector:
    """
    A class for performing object detection on images and videos using YOLOv8.

    Args:
    ------------
        model_file (str): Path to the YOLOv8 model file or yolo model variant name in ths format: [variant].pt
        labels (list[str], optional): A list of class labels for the model. If None, uses the default labels from the model file.
        classes (list[int], optional): The classes we want to detect, use it to exclude certain classes from being detected,
            by default all classes in labels are detectable.
        conf (float, optional): Minimum confidence threshold for object detection.
        iou (float, optional): Minimum IOU threshold for non-max suppression.

    Attributes:
    --------------
        classes (list[str]): A list of class labels for the model ( a Dict is also acceptable).
        conf (float): Minimum confidence threshold for object detection.
        iou (float): Minimum IOU threshold for non-max suppression.
        model (YOLO): The YOLOv8 model used for object detection.
        model_name (str): The name of the YOLOv8 model file (without the .pt extension).

    Methods :
    -------------
        default_display: Returns a default display (ultralytics plot implementation) of the object detection results.
        custom_display: Returns a custom display of the object detection results.
        predict_video: Predicts objects in a video and saves the results to a file.
        predict_img: Predicts objects in an image and returns the detection results.

    """

    def __init__(self, model_file = 'yolov8n.pt', labels= None, classes = None, conf = 0.25, iou = 0.45 ):

        self.classes = classes
        self.conf = conf
        self.iou = iou

        self.model = YOLO(model_file)
        self.model_name = model_file.split('.')[0]
        self.results = None

        # if no labels are provided then use default COCO names
        if labels == None:
            self.labels = self.model.names
        else:
            self.labels = labels

    def predict_img(self, img, verbose=True):
        """
        Runs object detection on a single image.

        Parameters
        ----------
            img (numpy.ndarray): Input image to perform object detection on.
            verbose (bool): Whether to print detection details.

        Returns:
        -----------
            'ultralytics.yolo.engine.results.Results': A YOLO results object that contains
             details about detection results :
                    - Class IDs
                    - Bounding Boxes
                    - Confidence score
                    ...
        (pls refer to https://docs.ultralytics.com/reference/results/#results-api-reference for results API reference)

        """

        # Run the model on the input image with the given parameters
        results = self.model(img, classes=self.classes, conf=self.conf, iou=self.iou, verbose=verbose)

        # Save the original image and the results for further analysis if needed
        self.orig_img = img
        self.results = results[0]

        # Return the detection results
        return results[0]



    def default_display(self, show_conf=True, line_width=None, font_size=None,
                        font='Arial.ttf', pil=False, example='abc'):
        """
        Displays the detected objects on the original input image.

        Parameters
        ----------
        show_conf : bool, optional
            Whether to show the confidence score of each detected object, by default True.
        line_width : int, optional
            The thickness of the bounding box line in pixels, by default None.
        font_size : int, optional
            The font size of the text label for each detected object, by default None.
        font : str, optional
            The font type of the text label for each detected object, by default 'Arial.ttf'.
        pil : bool, optional
            Whether to return a PIL Image object, by default False.
        example : str, optional
            A string to display on the example bounding box, by default 'abc'.

        Returns
        -------
        numpy.ndarray or PIL Image
            The original input image with the detected objects displayed as bounding boxes.
            If `pil=True`, a PIL Image object is returned instead.

        Raises
        ------
        ValueError
            If the input image has not been detected by calling the `predict_img()` method first.
        """
        # Check if the `predict_img()` method has been called before displaying the detected objects
        if self.results is None:
            raise ValueError('No detected objects to display. Call predict_img() method first.')

        # Call the plot() method of the `self.results` object to display the detected objects on the original image
        display_img = self.results.plot(show_conf, line_width, font_size, font, pil, example)

        # Return the displayed image
        return display_img



    def custom_display(self, colors, show_cls = True, show_conf = True):
        """
        Custom display method that draws bounding boxes and labels on the original image,
        with additional options for showing class and confidence information.

        Parameters:
        -----------
        colors : list
            A list of tuples specifying the color of each class.
        show_cls : bool, optional
            Whether to show class information in the label text. Default is True.
        show_conf : bool, optional
            Whether to show confidence information in the label text. Default is True.

        Returns:
        --------
        numpy.ndarray
            The image with bounding boxes and labels drawn on it.
        """

        img = self.orig_img
        # calculate the bounding box thickness based on the image width and height
        bbx_thickness = (img.shape[0] + img.shape[1]) // 450

        for box in self.results.boxes:
            textString = ""

            # Extract object class and confidence score
            score = box.conf.item() * 100
            class_id = int(box.cls.item())

            x1 , y1 , x2, y2 = np.squeeze(box.xyxy.numpy()).astype(int)

            # Print detection info
            if show_cls:
                textString += f"{self.labels[class_id]}"

            if show_conf:
                textString += f" {score:,.2f}%"

            # Calculate font scale based on object size
            font = cv2.FONT_HERSHEY_COMPLEX
            fontScale = (((x2 - x1) / img.shape[0]) + ((y2 - y1) / img.shape[1])) / 2 * 2.5
            fontThickness = 1
            textSize, baseline = cv2.getTextSize(textString, font, fontScale, fontThickness)

            # Draw bounding box, a centroid and label on the image
            img = cv2.rectangle(img, (x1,y1), (x2,y2), colors[class_id], bbx_thickness)
            center_coordinates = ((x1 + x2)//2, (y1 + y2) // 2)

            img =  cv2.circle(img, center_coordinates, 5 , (0,0,255), -1)

             # If there are no details to show on the image
            if textString != "":
                if (y1 < textSize[1]):
                    y1 = y1 + textSize[1]
                else:
                    y1 -= 2
                # show the details text in a filled rectangle
                img = cv2.rectangle(img, (x1, y1), (x1 + textSize[0] , y1 -  textSize[1]), colors[class_id], cv2.FILLED)
                img = cv2.putText(img, textString ,
                    (x1, y1), font,
                    fontScale,  (0, 0, 0), fontThickness, cv2.LINE_AA)

        return img


    def predict_video(self, video_path, save_dir, save_format="avi", display='custom', verbose=True, **display_args):
        """Runs object detection on each frame of a video and saves the output to a new video file.

        Args:
        ----------
            video_path (str): The path to the input video file.
            save_dir (str): The path to the directory where the output video file will be saved.
            save_format (str, optional): The format for the output video file. Defaults to "avi".
            display (str, optional): The type of display for the detection results. Defaults to 'custom'.
            verbose (bool, optional): Whether to print information about the video file and output file. Defaults to True.
            **display_args: Additional arguments to be passed to the display function.

        Returns:
        ------------
            None
        """
        # Open the input video file
        cap = cv2.VideoCapture(video_path)

        # Get the name of the input video file
        vid_name = os.path.basename(video_path)

        # Get the dimensions of each frame in the input video file
        width = int(cap.get(3))  # get `width`
        height = int(cap.get(4))  # get `height`

        # Create the directory for the output video file if it does not already exist
        if not os.path.isdir(save_dir):
            os.makedirs(save_dir)

        # Set the name and path for the output video file
        save_name = self.model_name + ' -- ' + vid_name.split('.')[0] + '.' + save_format
        save_file = os.path.join(save_dir, save_name)

        # Print information about the input and output video files if verbose is True
        if verbose:
            print("----------------------------")
            print(f"DETECTING OBJECTS IN : {vid_name} : ")
            print(f"RESOLUTION : {width}x{height}")
            print('SAVING TO :' + save_file)

        # Define an output VideoWriter object
        out = cv2.VideoWriter(save_file,
                              cv2.VideoWriter_fourcc(*"MJPG"),
                              30, (width, height))

        # Check if the input video file was opened correctly
        if not cap.isOpened():
            print("Error opening video stream or file")

        # Read each frame of the input video file
        while cap.isOpened():
            ret, frame = cap.read()

            # If the frame was not read successfully, break the loop
            if not ret:
                print("Error reading frame")
                break

            # Run object detection on the frame and calculate FPS
            beg = time.time()
            results = self.predict_img(frame, verbose=False)
            if results is None:
                print('***********************************************')
            fps = 1 / (time.time() - beg)

            # Display the detection results
            if display == 'default':
                frame = self.default_display(**display_args)
            elif display == 'custom':
                frame == self.custom_display(**display_args)

            # Display the FPS on the frame
            frame = cv2.putText(frame, f"FPS : {fps:,.2f}",
                                (5, 15), cv2.FONT_HERSHEY_COMPLEX,
                                0.5, (0, 0, 255), 1, cv2.LINE_AA)

            # Write the frame to the output video file
            out.write(frame)

            # Exit the loop if the 'q' button is pressed
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        # After the loop release the cap and video writer
        cap.release()
        out.release()

class YOLOv8_ObjectCounter(YOLOv8_ObjectDetector):
    """
    A class for counting objects in images or videos using the YOLOv8 Object Detection model.

    Attributes:
    -----------
    model_file : str
        The filename of the YOLOv8 object detection model.
    labels : list or None
        The list of labels for the object detection model. If None, the labels will be loaded from the model file.
    classes : list or None
        The list of classes to detect. If None, all classes will be detected.
    conf : float
        The confidence threshold for object detection.
    iou : float
        The Intersection over Union (IoU) threshold for object detection.
    track_max_age : int
        The maximum age (in frames) of a track before it is deleted.
    track_min_hits : int
        The minimum number of hits required for a track to be considered valid.
    track_iou_threshold : float
        The IoU threshold for matching detections to existing tracks.

    Methods:
    --------
    predict_img(img, verbose=True)
        Predicts objects in a single image and counts them.
    predict_video(video_path, save_dir, save_format="avi", display='custom', verbose=True, **display_args)
        Predicts objects in a video and counts them.

    """

    def __init__(self, model_file = 'yolov8n.pt', labels= None, classes = None, conf = 0.25, iou = 0.45,
                 track_max_age = 45, track_min_hits= 15, track_iou_threshold = 0.3 ):

        super().__init__(model_file , labels, classes, conf, iou)

        self.track_max_age = track_max_age
        self.track_min_hits = track_min_hits
        self.track_iou_threshold = track_iou_threshold




    def predict_video(self, video_path, save_dir, save_format = "avi",
                      display = 'custom', verbose = True, **display_args):

        """
    Runs object detection on a video file and saves the output as a new video file.

    Args:
        video_path (str): Path to the input video file.
        save_dir (str): Path to the directory where the output video file will be saved.
        save_format (str, optional): Format of the output video file. Defaults to "avi".
        display (str, optional): Type of display to use for object detection results. Options are "default" or "custom".
                                Defaults to "custom".
        verbose (bool, optional): If True, prints information about the input and output video files. Defaults to True.
        **display_args (dict, optional): Additional arguments to pass to the display function.

    Returns:
        None
        """
        cap = cv2.VideoCapture(video_path)
        # Get video name
        vid_name = os.path.basename(video_path)


        # Get frame dimensions and print information about input video file
        width  = int(cap.get(3) )  # get `width`
        height = int(cap.get(4) )  # get `height`

        if not os.path.isdir(save_dir):
            os.makedirs(save_dir)

        save_name = self.model_name + ' -- ' + vid_name.split('.')[0] + '.' + save_format
        save_file = os.path.join(save_dir, save_name)

        if verbose:
            print("----------------------------")
            print(f"DETECTING OBJECTS IN : {vid_name} : ")
            print(f"RESOLUTION : {width}x{height}")
            print('SAVING TO :' + save_file)

        # define an output VideoWriter  object
        out = cv2.VideoWriter(save_file,
                            cv2.VideoWriter_fourcc(*"MJPG"),
                            30,(width,height))

        # Check if the video is opened correctly
        if not cap.isOpened():
            print("Error opening video stream or file")

        # Initialize object tracker
        tracker = sort.Sort(max_age = self.track_max_age, min_hits= self.track_min_hits ,
                            iou_threshold = self.track_iou_threshold)

        # Initialize variables for object counting
        totalCount = []
        currentArray = np.empty((0, 5))


        # Read the video frames
        while cap.isOpened():

            detections = np.empty((0, 5))
            ret, frame = cap.read()

            # If the frame was not read successfully, break the loop
            if not ret:
                print("Error reading frame")
                break

            # Run object detection on the frame and calculate FPS
            beg = time.time()
            results = self.predict_img(frame, verbose = False)
            if results == None:
                print('***********************************************')
            fps = 1 / (time.time() - beg)
            for box in results.boxes:
                score = box.conf.item() * 100
                class_id = int(box.cls.item())

                x1 , y1 , x2, y2 = np.squeeze(box.xyxy.numpy()).astype(int)

                currentArray = np.array([x1, y1, x2, y2, score])
                detections = np.vstack((detections, currentArray))

            # Update object tracker
            resultsTracker = tracker.update(detections)
            for result in resultsTracker:
                #print(type(result))

                # Get the tracker results
                x1, y1, x2, y2, id = result
                x1, y1, x2, y2, id = int(x1), int(y1), int(x2), int(y2), int(id)
                #print(result)

                # Display current objects IDs
                w, h = x2 - x1, y2 - y1
                cx, cy = x1 + w // 2, y1 + h // 2
                id_txt = f"ID: {str(id)}"
                cv2.putText(frame, id_txt, (cx, cy), 4, 0.5, (0, 0, 255), 1)

                # if we haven't seen aprticular object ID before, register it in a list
                if totalCount.count(id) == 0:
                    totalCount.append(id)

            # Display detection results
            if display == 'default':
                frame = self.default_display(**display_args)

            elif display == 'custom':
                frame == self.custom_display( **display_args)

            # Display FPS on frame
            frame = cv2.putText(frame,f"FPS : {fps:,.2f}" ,
                                (5,55), cv2.FONT_HERSHEY_COMPLEX,
                            0.5,  (0,255,255), 1, cv2.LINE_AA)

            # Display Counting results
            count_txt = f"TOTAL COUNT : {len(totalCount)}"
            frame = cv2.putText(frame, count_txt, (5,45), cv2.FONT_HERSHEY_COMPLEX, 2, (0, 0, 255), 2)


            # append frame to the video file
            out.write(frame)

            # the 'q' button is set as the
            # quitting button you may use any
            # desired button of your choice

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        # After the loop release the cap
        cap.release()
        out.release()
        print(len(totalCount))
        print(totalCount)